# Assignment 7

In this assignment, we'll use our two variational inference algorithms, and compare their speed and output to MCMC.

## Instructions

Please complete this Jupyter notebook and **don't** convert it to a `.py` file. Upload this notebook, along with any `.stan` files and any data sets as a `zip` file to Gradescope. Your work will be manually graded by our TA. 


In [2]:
import pandas as pd
import numpy as np
import os
from cmdstanpy import CmdStanModel
import matplotlib.pyplot as plt

### Problem 1: A Derivation

In lecture it was mentioned that the KL-divergence is an intractable quantity. However, the **Evidence Lower Bound (ELBO)**  is usually not. Your goal is to show that this.

Show

\begin{align*}
\text{KL}(g || p) 
&= - \mathbb{E}_{g}\left[ \log\frac{  L(y  \mid \theta) \pi(\theta \mid y)}{g(\theta \mid \phi)} \right] +  \text{constant}  
\end{align*}

Or in other words, show that  the ELBO--the expectation without the negative sign on the right hand side--doesn't involve the unknown normalizing constant $p(y)$.

### Problem 2: Multinomial Regression

Bayes' theorem looks a little different with supervised learning. First, data is split up into predictors (i.e. $x$) and dependent variables (call them $y$). In this example, we will also call our parameters $\beta$, instead of $\theta$. 

So, Bayes' rule will look like this

$$
\pi(\beta \mid y, x) \propto L(y \mid \beta, x) \pi(\beta)
$$


1.

Prove the above using rules of conditioning and any other required assumptions. Attach a screenshot of your deriation to this notebook. 



Previously, we modeled categorical output with a multinomial distribution. Our previous notation for one row of data was as follows:

Let $y = (y_1, y_2, \ldots, y_k)$ be a vector of counts.

We assume that there is a known total count (which means $\sum_i y_i = n$). 

A special case of the multinomial distribution--when $\sum_i y_i$ is $1$--is the **Categorical Distribution**. 

This was just like the distinction between Bernoulli and Binomial random variables. A Bernoulli is when there is one trial, and a Binomial is when there are many. Here it is the same. A Categorical random variable is when there is only one trial, and a multinomial is more general and allows for multiple trails. Each trial here, though, each categorical realization, can have multiple outcomes!

Because there is only one trial for each categorical realization, there are multiple ways to represent it ina program. 

One way is the **one-hot-encoding**. It matches the notation that we were using above. We could represent each realization as a length $k$ vector, and only allow one of the elements to be $1$ (or "hot").

A more memory-efficient way is to just store the index/number of the category that took place (e.g. $3$). That's what we are doing with our data set. This way, we only need one column to store the dependent data.

2.

Suppose each message on a social media platform can have four possible sentiments: "outrage", "joking", "intrigued" and "bored". Suppose you are looking at three tweets. The first is outrage, the second is bored, and the third is intrigued. 

Write out two ways that you could represent this data. Describe your category ordering and index labeling. Does your counting start from $1$ or $0$. Are your categories ordered alphabetically? Use the one-hot encoding as well as the simple category labels.  Attach a screenshot of your deriation to this notebook. 

Previously, our data were so simplistic that we assumed each observation--each row of data (usually)--was from the same distribution. In other words, the probabilities of each "bucket" stay the same. We wrote it like this

Let $\theta = (\theta_1, \theta_2, \ldots, \theta_k)$ be the probabilities of any trial resulting in each of the $k$ outcomes. 

We also assume the only possible outcomes are these $k$ outcomes so $\sum_i \theta_i = 1$.

One way we can generalize this model is to allow the probabilities to vary. We could let them be affected by predictors. That's what we'll do in this homework. We will take our predictors, multiple them with some parameters, then use a nonlinear function to get the probabilities for each bucket. 

**Please note that we will describe the details of this model in more detail in a future module!**



Here is our high-level notation:

 - $N$: the number of observations/rows we have
 - $i$: the row number...goes from $1$ to $N$
 - $K$: number of categories/buckets we try to predict
 - $y_i$: the categorical dependent variable in row $i$ (it can be $1, \ldots, K$)
 - $y$ is the set of all $y_1, \ldots, y_N$
 - $D$: the number of predictors we can use to inform bucket probabilities
 - $x_i$: the row of predictor information. It has length $D$
 - $x$ is the matrix of all row predictors. It has shape $N \times D$

To drive this home, let's look at our specific data set, and try to make it look like this notation. 

In [5]:
d = pd.read_csv("SoftmaxRegData1.csv")
y = d['Y']
x = d[['X1','X2']].assign(intercept=1)
print(y.head())
print("\n\nunique y values: ", np.unique(y), "\n\n")
x.head()

0    2
1    1
2    3
3    3
4    3
Name: Y, dtype: int64


unique y values:  [1 2 3 4] 




,X1,X2,intercept
0,-0.087147,-1.081342,1
1,-0.722566,-1.583863,1
2,0.179190,0.971790,1
3,-1.159752,0.502624,1
4,-0.727118,1.375704,1


Our parameters of the model are "weights" that we weight predictors by. 

For parsimony, we use the same weights for each row of data. 

Also, each column of $\beta$ will take data an give a probability output *for each possible outcome.* 

This makes the weight matrix $\beta$ a $D \times K$. For us, that's $3 \times 4$.

We can write our likelihood in a few different ways.

$$
L(y \mid x, \beta) = \prod_{i=1}^N L(y_i \mid x_i, \beta)
$$
and
$$
L(y_i \mid x_i, \beta) = \text{Categorical}(\text{softmax}(x_i ^\intercal \beta) )
$$

**We will discuss the softmax function more later.** Suffice it to say that $\text{softmax}(x_i ^\intercal \beta)$ is a column vector of probabilities. Each probability describes the chances of each possible outome of the dependent variable $y_i$. Further, these probabilities depend on data for that particular row. They were kind of like our $\theta$s from before, but they now depend on the data we have in a particular row.

2.

I have provided a `.stan` file called `multinomial_regression.stan` that describes the model above. Take a look at it, and observe how it maps to the above description of the model. After you are comfortable with it, build your model into an object called `model`.


In [68]:
model_code = ...
model = ...

3. 

Use NUTS the MCMC algorithm to draw samples from this posterior. 

Do all the $\hat{R}$s look good for each parameter? What are your parameter estimates? What are 90% credible intervals for those parameter estimates? Don't worry about writing too much...just call `.summary()` in a cell.

Which parameters are very correlated (a posteriori) and how so? Why do you think is this the case? Justify your answer. Use `pd.plotting.scatter_matrix()` to visualize all the pairwise correlations of the posterior samples. 

In [69]:
our_data = ...
fit = ...

4. 

Use Pathfinder VI algorithm to obtain a posterior approximation. 

Are the parameter estimates close to the MCMC parameter estimates? 

Was it much faster than the MCMC sampler?

Which parameters are very correlated (a posteriori) and how so? Why do you think is this the case? Use `pd.plotting.scatter_matrix()` to visualize all the pairwise correlations of the posterior samples. 

In [70]:
fit2 = ...

5. 

Use the ADVI algorithm to obtain a posterior approximation. 

Are the parameter estimates close to the pathfinder's estimates?

Was it much faster than the MCMC sampler?

Which parameters are very correlated (a posteriori) and how so? Why do you think is this the case? Use `pd.plotting.scatter_matrix()` to visualize all the pairwise correlations of the posterior samples. 

In [71]:
fit3 = ...

6.

Describe two different approaches on how to simulate from the posterior predictive distribution. Why is this more interesting than simulating from the posterior preditive of our other models? 

Hint: did we make any modeling assumptions for the predictor data?